In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../..')

import pandas as pd
import numpy as np

from functools import reduce 
from utils.use_regression import (create_formula, convert_results_to_df, run_regression)

### 1. Import Data

#### HMDA Data that contains all dummy variables

In [3]:
hmda19_df = pd.read_csv('../../data/hmda_lar/cleaned_data/3_hmda2019_regressiondata_210823.csv',
                        dtype = {'app_credit_model': str, 'metro_code': str})

hmda19_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2433071 entries, 0 to 2433070
Data columns (total 63 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   black                         float64
 1   latino                        float64
 2   asian_cb                      float64
 3   native                        float64
 4   race_na                       float64
 5   no_coapplicant                float64
 6   na_coapplicant                float64
 7   female                        float64
 8   sex_na                        float64
 9   less_than25                   float64
 10  between25_34                  float64
 11  between45_54                  float64
 12  between55_64                  float64
 13  older_than65                  float64
 14  age_na                        float64
 15  income_log                    float64
 16  loan_log                      float64
 17  property_value_ratio          float64
 18  less30yrs_mortgage    

#### Metro Data

In [4]:
metros_df = pd.read_csv('../../data/census_data/county_to_metro_crosswalk/clean/all_counties_210804.csv',
                        dtype = str)

metros_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3225 entries, 0 to 3224
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   fips_state_code   3224 non-null   object
 1   fips_county_code  3224 non-null   object
 2   county_name       3220 non-null   object
 3   state_name        3220 non-null   object
 4   cbsa_code         1916 non-null   object
 5   cbsa_title        1916 non-null   object
 6   csa_code          1256 non-null   object
 7   csa_title         1256 non-null   object
 8   metro_type        3225 non-null   object
 9   metro_code        1916 non-null   object
 10  metro_name        1916 non-null   object
 11  metro_pop         1915 non-null   object
 12  metro_percentile  3225 non-null   object
 13  metro_type_def    3225 non-null   object
dtypes: object(14)
memory usage: 352.9+ KB


### 2. Reshape metro data

#### Data is at the county level, grouping by metros

In [5]:
metros_df2 = pd.DataFrame(metros_df.groupby(by = ['metro_code', 'metro_name', 'metro_type', 'metro_pop'],
                          dropna = False).size()).reset_index().rename(columns = {0: 'count'}).\
                          drop(columns = {'count'})

metros_df2['metro_pop'] = pd.to_numeric(metros_df2['metro_pop'])

### 3. Filter HMDA data
#### Filtering those NA's that are less than one percent of the column. These values break at the metro level

- 0: Yes
- 1: No

In [6]:
hmda19_df['na_coapplicant'].value_counts(dropna = False, normalize = True) * 100

1.0    99.820145
0.0     0.179855
Name: na_coapplicant, dtype: float64

In [7]:
hmda19_df['age_na'].value_counts(dropna = False, normalize = True) * 100

1.0    99.981957
0.0     0.018043
Name: age_na, dtype: float64

In [8]:
hmda19_df['lender_na'].value_counts(dropna = False, normalize = True) * 100

1.0    99.730135
0.0     0.269865
Name: lender_na, dtype: float64

In [9]:
hmda19_df2 = hmda19_df[(hmda19_df['na_coapplicant'] != 0) & (hmda19_df['age_na'] != 0) &\
                       (hmda19_df['lender_na'] != 0)]

#### Filter out Metros with no code

In [10]:
hmda19_df3 = hmda19_df2[(hmda19_df2['metro_code'].notnull())]

print(len(hmda19_df3))

2357940


### 4. Setup independent variables

In [11]:
independent_vars = ['black', 'latino', 'native', 'asian_cb', 'race_na',
                    'female', 'sex_na',
                    'no_coapplicant', 
                    'younger_than_34', 'older_than_55', 
                    'income_log', 
                    'loan_log', 
                    'property_value_ratio', 
                    'not30yr_mortgage', 
                    'equifax', 'experian', 'other_model', 'more_than_one', 'model_na',
                    'dti_manageable', 'dti_unmanageable', 'dti_struggling',
                    'combined_loan_to_value_ratio',
                    'low_lmi', 'moderate_lmi', 'middle_lmi',
                    'credit_union', 'independent',
                    'lar_count',
                    'non_desktop', 'aus_na', 
                    'white_cat2', 'white_cat3', 'white_cat4']

continuous_vars = ['income_log', 'loan_log', 'combined_loan_to_value_ratio', 'lar_count', 'prop_zscore']

### 5. Get variable count for each metro

In [12]:
metros = hmda19_df3['metro_code'].unique()

print('Number of metros: ' + str(len(metros)))

Number of metros: 959


#### Count all the independent variables for each metro

In [13]:
### Excluding continous variables from the counting
independent_vars2 = [var for var in independent_vars if var not in continuous_vars]

metro_var_holder = []

for independent_var in independent_vars2:
    index_values = []
    index_values.extend(('metro_code', independent_var))
    
    metro_var_df = pd.pivot_table(hmda19_df3, index = index_values, columns = ['loan_outcome'], 
                                  values = ['denied'], aggfunc = 'count', fill_value = 0).reset_index()
    
    metro_var_df.columns = metro_var_df.columns.droplevel(0)
    metro_var_df.columns.name = None
    metro_var_df.columns = ['metro_code', 'variable_flag', 'loan', 'denied']
    metro_var_df['variable_name'] = independent_var
    
    metro_var_holder.append(metro_var_df)
    
metro_varcount_df = pd.concat(metro_var_holder)
metro_varcount_df['metro_code'].nunique()

959

#### Add missing records to the variable count dataframe

In [14]:
metro_varcount_df2 = metro_varcount_df[(metro_varcount_df['variable_flag'] == 0)]
missing_rows_list = []

for metro in metros:
    metro_vars_df = metro_varcount_df2[(metro_varcount_df2['metro_code'] == metro)]
    metro_vars = metro_vars_df['variable_name'].unique()
    
    ### including the continous variables
    for reference_var in independent_vars:
        if reference_var not in metro_vars:
            missing_row = pd.DataFrame([[metro, 0, 0, 0, reference_var]], columns = ['metro_code', 
                                               'variable_flag', 'loan', 'denied', 'variable_name'])
            missing_rows_list.append(missing_row)
            
missing_rows_df = pd.concat(missing_rows_list)         
metro_varcount_df3 = metro_varcount_df2.append(missing_rows_df)

#### Find variable total count and percentages

In [16]:
metro_varcount_df3['total_count'] = metro_varcount_df3['loan'] + metro_varcount_df3['denied']

metro_varcount_df3['loan_pct'] = metro_varcount_df3['loan'].div(metro_varcount_df3['total_count']).multiply(100)

metro_varcount_df3['denied_pct'] = metro_varcount_df3['denied'].\
                                   div(metro_varcount_df3['total_count']).multiply(100)

### 6. Run regression for all metros

In [17]:
metro_analysis = []
i = 0

for metro in metros:
    print(str(i) + ': Metro: ' + metro)
    metro_df = hmda19_df3[(hmda19_df3['metro_code'] == metro)]
    metro_apps = len(metro_df)
    
    regression_formula = create_formula(independent_vars)
    model = run_regression(data = metro_df, formula = regression_formula)
    
    try:
        results = model.fit()
        info = results.mle_retvals['converged']
        
        results_df = convert_results_to_df(results)
        results_df.insert(0, 'metro_code', metro)
        results_df.insert(1, 'metro_apps', metro_apps)
        results_df.insert(2, 'psuedo_rsquare', results.prsquared)
        results_df['iteration_flag'] = info
        
    except:
        independent_nan_list = []
        for regression_var in independent_vars:
            results_dict = {'metro_code': metro, 'metro_apps': metro_apps, 'variable_name': regression_var, 
                            'standard_error': np.nan,  'z_value': np.nan, 'p_value': np.nan, 'odds_ratio': np.nan, 
                            'iteration_flag': np.nan, 'psuedo_rsquare': np.nan}
            
            non_results_df = pd.DataFrame([results_dict], columns = results_dict.keys())
            independent_nan_list.append(non_results_df)
            
        results_df = pd.concat(independent_nan_list)
        
    metro_analysis.append(results_df)
    i += 1
    
results_df2 = pd.concat(metro_analysis)

0: Metro: 35084
Optimization terminated successfully.
         Current function value: 0.203528
         Iterations 8
1: Metro: 35614
Optimization terminated successfully.
         Current function value: 0.260659
         Iterations 7
2: Metro: 35154
Optimization terminated successfully.
         Current function value: 0.212554
         Iterations 8
3: Metro: 39100
Optimization terminated successfully.
         Current function value: 0.191431
         Iterations 8
4: Metro: 45940
         Current function value: 0.218257
         Iterations: 35
5: Metro: 20700


/Users/emmanuel/.pyenv/versions/3.8.0/envs/redlining/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Optimization terminated successfully.
         Current function value: 0.216407
         Iterations 25
6: Metro: 48424
Optimization terminated successfully.
         Current function value: 0.270169
         Iterations 7
7: Metro: 33124
         Current function value: 0.338133
         Iterations: 35
8: Metro: 37964


/Users/emmanuel/.pyenv/versions/3.8.0/envs/redlining/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


         Current function value: 0.201493
         Iterations: 35
9: Metro: 12100


...

/Users/emmanuel/.pyenv/versions/3.8.0/envs/redlining/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


887: Metro: 25780
         Current function value: 0.209819
         Iterations: 35
888: Metro: 11660


/Users/emmanuel/.pyenv/versions/3.8.0/envs/redlining/lib/python3.8/site-packages/statsmodels/discrete/discrete_model.py:1810: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/Users/emmanuel/.pyenv/versions/3.8.0/envs/redlining/lib/python3.8/site-packages/statsmodels/discrete/discrete_model.py:1863: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))


         Current function value: inf
         Iterations: 35
889: Metro: 32180
         Current function value: inf
         Iterations: 35
890: Metro: 35900
         Current function value: inf
         Iterations: 35
891: Metro: 40260
         Current function value: 0.134813
         Iterations: 35
892: Metro: 15620
         Current function value: inf
         Iterations: 35
893: Metro: 39060
         Current function value: 0.190528
         Iterations: 35
894: Metro: 44980
         Current function value: inf
         Iterations: 35
895: Metro: 36700
         Current function value: 0.184804
         Iterations: 35
896: Metro: 33060
         Current function value: inf
         Iterations: 35
897: Metro: 34780
         Current function value: inf
         Iterations: 35
898: Metro: 11060
         Current function value: inf
         Iterations: 35
899: Metro: 34460
         Current function value: inf
         Iterations: 35
900: Metro: 28380
         Current function value: 0.00

954: Metro: 21860
         Current function value: 0.237304
         Iterations: 35
955: Metro: 48100
         Current function value: inf
         Iterations: 35
956: Metro: 13500
         Current function value: inf
         Iterations: 35
957: Metro: 17640
958: Metro: 27580


### 7. Join results with metros data
#### Is every variable accounted for:
- If a variables has less than 959, then it's missing in certain metros

In [18]:
var_used_check = pd.DataFrame(results_df2['variable_name'].value_counts(dropna = False)).reset_index()

var_used_check[(var_used_check['variable_name'] < 959)]

,index,variable_name
34,Intercept,250


#### Join with metro names and var count dataframes

In [19]:
results_df3 = pd.merge(results_df2, metros_df2, how = 'left', on = ['metro_code'])
results_df4 = pd.merge(results_df3, metro_varcount_df3, how = 'left',  on = ['metro_code', 'variable_name'])

### 8. Categorize Results

#### Filter for metros that don't produce any results
- Results Flag 1: No meaningful results, all columns shows up as NaN

In [20]:
results_df4.loc[((results_df4['psuedo_rsquare'].isnull()) & (results_df4['coefficient'].isnull()) & \
                 (results_df4['standard_error'] .isnull()) & (results_df4['z_value'].isnull()) & \
                 (results_df4['p_value'].isnull()) & (results_df4['odds_ratio'].isnull())), 
                 'results_flag'] = '1'

metros_no_results = results_df4[(results_df4['results_flag'] == '1')]['metro_code'].nunique()

all_metros = results_df4['metro_code'].nunique()

#### Metro Results Breakdown:
- 709 metros don't produce results
- 250 metros prodocues results

In [21]:
print('Percent of metros with no results: ' + str(((metros_no_results/all_metros) * 100)))

print('Number of metros that DON\'T produce results: ' + str(metros_no_results))

print('Number of metros that produce results: ' + str(results_df4[(results_df4['results_flag'] != '1')] \
                                                      ['metro_code'].nunique()))

Percent of metros with no results: 73.93117831074035
Number of metros that DON'T produce results: 709
Number of metros that produce results: 250


In [22]:
metros_size_df = pd.DataFrame(results_df4[(results_df4['results_flag'] == '1')].\
                 groupby(by = ['metro_code', 'metro_apps']).size()).reset_index().rename(columns = {0: 'count'})

metros_size_df['metro_apps'].describe()

count     709.000000
mean      473.320169
std       690.641226
min         1.000000
25%       120.000000
50%       235.000000
75%       502.000000
max      7148.000000
Name: metro_apps, dtype: float64

#### Filter for metros that produce the results, but the equation needs work
- Results Flag: 2, no meaningful results because of the equation 

In [23]:
results_df4.loc[(results_df4['psuedo_rsquare'] < .1) | (results_df4['iteration_flag'] == False),
                'results_flag'] = '2'

broken_results_metros = results_df4[(results_df4['results_flag'] == '2')]['metro_code'].nunique()

#### Metro Results Breakdown:
- 121 metros with unreliable results, not enoght variance in variables. 

In [24]:
print('Number of metros with broken results: ' + str(broken_results_metros))
print('Percentage of metros with no results: ' + str((broken_results_metros/all_metros) * 100))

Number of metros with broken results: 122
Percentage of metros with no results: 12.721584984358708


#### Filter for metros where the equation is valid
- Results flag 3: valid results

In [25]:
results_df4.loc[(results_df4['psuedo_rsquare'] >= .1) & (results_df4['iteration_flag'] == True), 
                'results_flag'] = '3'

valid_results_metros = results_df4[(results_df4['results_flag'] == '3')]['metro_code'].nunique()

#### Metro Results Breakdown:
- 128 metros with reliable results.

In [26]:
print('Number of metros with valid results: ' + str(valid_results_metros))
print('Percentage of metros with valid results: ' + str((valid_results_metros/all_metros) * 100))

Number of metros with valid results: 128
Percentage of metros with valid results: 13.347236704900938


#### Overall metros breakdown
- 1: No Results
- 2: Variable Issues
- 3: Results

In [27]:
metro_results_df = pd.DataFrame(results_df4.groupby(by = ['results_flag', 'metro_code']).size()).reset_index().\
                   rename(columns = {0: 'count'})

metro_results_df['results_flag'].value_counts(dropna = False)

1    709
3    128
2    122
Name: results_flag, dtype: int64

### 9. Assessing the variables of valid metros
#### Variables where the p-value and z-value are missing
- 63 records

In [28]:
results_df4.loc[(results_df4['results_flag'] == '3') & (results_df4['z_value'].isnull()) &\
                (results_df4['p_value'].isnull()), 'variable_check'] = '1'

results_df4['variable_check'].value_counts(dropna = False)

NaN    32793
1         63
Name: variable_check, dtype: int64

#### Metros with a varibale that has a missing p-value
- 17 metros that are valid but there's missing p-values in them

In [29]:
missing_pvalues_metros_df = results_df4[(results_df4['results_flag'] == '3') & \
                                        (results_df4['variable_check'] == '1')]

print(missing_pvalues_metros_df['metro_code'].nunique())
missing_pvalues_metros_df['metro_name'].value_counts(dropna = False)

17


Lansing-East Lansing, MI                         12
Rockingham County-Strafford County, NH            6
Yuba City, CA                                     6
Mount Vernon-Anacortes, WA                        6
Madison, WI                                       4
Boise City, ID                                    4
Kennewick-Richland, WA                            4
Evansville, IN-KY                                 3
El Paso, TX                                       3
Knoxville, TN                                     2
Portland-Vancouver-Hillsboro, OR-WA               2
Duluth, MN-WI                                     2
Ocala, FL                                         2
Wilmington, NC                                    2
Reading, PA                                       2
Myrtle Beach-Conway-North Myrtle Beach, SC-NC     2
Ann Arbor, MI                                     1
Name: metro_name, dtype: int64

#### Variables that are not statistcially significant
- 2693 records

In [30]:
results_df4.loc[(results_df4['results_flag'] == '3') & (results_df4['p_value'] >= .05),
                'variable_check'] = '2'

results_df4['variable_check'].value_counts(dropna = False)

NaN    30100
2       2693
1         63
Name: variable_check, dtype: int64

#### Filter for variables that are statistcially significant but need more applications
- Variable Check 3: 448 records

In [31]:
results_df4.loc[(results_df4['results_flag'] == '3') & (results_df4['p_value'] < .05) &\
                (results_df4['total_count'] < 75), 'variable_check'] = '3'

results_df4['variable_check'].value_counts(dropna = False)

NaN    29652
2       2693
3        448
1         63
Name: variable_check, dtype: int64

#### Filter those records that are statistcially signicant but no disparity
- Variable Check 4: 503 records

In [32]:
results_df4.loc[(results_df4['results_flag'] == '3') & (results_df4['p_value'] < .05) &\
                (results_df4['total_count'] >= 75) & (results_df4['odds_ratio'] < 1.45), 
                'variable_check'] = '4'

results_df4['variable_check'].value_counts(dropna = False)

NaN    29149
2       2693
4        503
3        448
1         63
Name: variable_check, dtype: int64

#### Filter those records that are statistcially signicant with a disparity
- Variable Check 5: 677 records

In [33]:
results_df4.loc[(results_df4['results_flag'] == '3') & (results_df4['p_value'] < .05) &\
                (results_df4['total_count'] >= 75) & (results_df4['odds_ratio'] >= 1.45), 
                'variable_check'] = '5'

results_df4['variable_check'].value_counts(dropna = False)

NaN    28472
2       2693
5        677
4        503
3        448
1         63
Name: variable_check, dtype: int64

### 10.  Breakdown of statistcially valid metros for race and ethnicity
- 96 NaN are all Intercept variables

In [34]:
results_df4[(results_df4['results_flag'] == '3')]['variable_check'].value_counts(dropna = False)

2      2693
5       677
4       503
3       448
NaN      96
1        63
Name: variable_check, dtype: int64

#### Statistically Valid Metros Race and Ethnicity Breakdown:
- 2: Not statistically significant -- 295 records
- 5: Statistically significant disparity -- 166 records
- 3: Not enough applications -- 25 records
- 4: Small disparities –– 23 records
- 1: Missing p-values -- 3 records

In [35]:
races = ['black', 'latino', 'asian_cb', 'native']

results_df5 = results_df4[(results_df4['results_flag'] == '3') & (results_df4['variable_name'].isin(races))]

results_df5['variable_check'].value_counts(dropna = False)

2    295
5    166
3     25
4     23
1      3
Name: variable_check, dtype: int64

In [36]:
results_df5['metro_code'].nunique()

128

#### Breakdown of 128 Metros

In [37]:
metro_results = pd.DataFrame(results_df5.groupby(by = ['metro_code', 'variable_check']).size()).reset_index().\
                rename(columns = {0: 'count'})

#### Number of metros with no reliable results:

In [38]:
reliable_metros_df = metro_results[(metro_results['variable_check'] == '5') |\
                                   (metro_results['variable_check'] == '4')]

reliable_metros = reliable_metros_df['metro_code'].unique()

#### Number of metros where all racial and ethnic variables are not statistically significant
- 21 metros

In [39]:
print(metro_results[~(metro_results['metro_code'].isin(reliable_metros)) & \
                     (metro_results['count'] == 4)]['metro_code'].nunique())

metro_results[~(metro_results['metro_code'].isin(reliable_metros)) & \
               (metro_results['count'] == 4)]['variable_check'].unique()

21


array(['2'], dtype=object)

#### Number of metros where all racial and ethnic variables are not reliable
- 16 metros
- Because not statistically significant, not enough applications, or missing a p-value

In [40]:
print(metro_results[~(metro_results['metro_code'].isin(reliable_metros)) & \
                     (metro_results['count'] < 4)]['metro_code'].nunique())

metro_results[~(metro_results['metro_code'].isin(reliable_metros)) & \
               (metro_results['count'] < 4)]['variable_check'].unique()

16


array(['2', '3', '1'], dtype=object)

#### Number of metros with valid results
- 91 metros

In [41]:
valid_results_df = metro_results[(metro_results['variable_check'] == '5') |\
                              (metro_results['variable_check'] == '4')]

valid_results_df['metro_code'].nunique()

91

#### Metros with at least one disparity:
- 89 metros

In [42]:
valid_results_df[(valid_results_df['variable_check'] == '5')]['metro_code'].nunique()

89

#### Metros where the only valid result is a small disparity
- 2 metros

In [43]:
disparity_metro = valid_results_df[(valid_results_df['variable_check'] == '5')]['metro_code'].unique()

valid_results_df[~(valid_results_df['metro_code'].isin(disparity_metro))]['metro_code'].nunique()

2

### 11. Places with small disparities

In [44]:
print(results_df5[(results_df5['variable_check'] == '4')]['metro_name'].nunique())

results_df5[(results_df5['variable_check'] == '4')]['metro_name'].value_counts(dropna = False)

18


New York-Jersey City-White Plains, NY-NJ     2
Phoenix-Mesa-Chandler, AZ                    2
Houston-The Woodlands-Sugar Land, TX         2
Orlando-Kissimmee-Sanford, FL                2
Las Vegas-Henderson-Paradise, NV             2
San Diego-Chula Vista-Carlsbad, CA           1
Seattle-Bellevue-Kent, WA                    1
Tampa-St. Petersburg-Clearwater, FL          1
Austin-Round Rock-Georgetown, TX             1
Nassau County-Suffolk County, NY             1
Newark, NJ-PA                                1
Chicago-Naperville-Evanston, IL              1
Kansas City, MO-KS                           1
Fort Lauderdale-Pompano Beach-Sunrise, FL    1
Los Angeles-Long Beach-Glendale, CA          1
Portland-Vancouver-Hillsboro, OR-WA          1
New Brunswick-Lakewood, NJ                   1
Charlotte-Concord-Gastonia, NC-SC            1
Name: metro_name, dtype: int64

#### Places where the only reliable results are small disparities

In [45]:
small_disparities = valid_results_df[~(valid_results_df['metro_code'].isin(disparity_metro))]['metro_code']

results_df5[(results_df5['metro_code'].isin(small_disparities)) & (results_df5['variable_check'] == '4')]\
[['metro_name', 'variable_name', 'p_value', 'odds_ratio', 'variable_check']]

,metro_name,variable_name,p_value,odds_ratio,variable_check
524,"Fort Lauderdale-Pompano Beach-Sunrise, FL",black,0.000018,1.433741,4
5306,"Las Vegas-Henderson-Paradise, NV",black,0.002907,1.415229,4
5309,"Las Vegas-Henderson-Paradise, NV",asian_cb,0.019246,1.217433,4


#### Smallest disparities overall

In [46]:
results_df5[(results_df5['variable_check'] == '4')][['metro_name', 'variable_name', 'p_value', 'odds_ratio', 
                                                     'variable_check']].sort_values(by = ['odds_ratio']).head(5)

,metro_name,variable_name,p_value,odds_ratio,variable_check
2329,"Los Angeles-Long Beach-Glendale, CA",latino,0.016634,1.153441,4
39,"New York-Jersey City-White Plains, NY-NJ",asian_cb,0.000722,1.175773,4
353,"Nassau County-Suffolk County, NY",asian_cb,0.047683,1.207980,4
5309,"Las Vegas-Henderson-Paradise, NV",asian_cb,0.019246,1.217433,4
74,"New Brunswick-Lakewood, NJ",asian_cb,0.020274,1.238206,4


#### Largest disparities overall

In [47]:
results_df5[(results_df5['variable_check'] == '5')][['metro_name', 'variable_name', 'p_value', 'odds_ratio', 
                                                     'variable_check']].sort_values(by = ['odds_ratio'],
                                                                                   ascending = False).head(5)

,metro_name,variable_name,p_value,odds_ratio,variable_check
12408,"Waco, TX",latino,0.003080,3.291675,5
18922,"Fayetteville-Springdale-Rogers, AR",black,0.007222,3.098998,5
4240,"Port St. Lucie, FL",asian_cb,0.000520,3.086573,5
9397,"Rockingham County-Strafford County, NH",asian_cb,0.000215,2.962371,5
11143,"Dayton-Kettering, OH",black,0.000306,2.882675,5


### 12. Breakdown of metros with disparities
#### Looking at 10 most populous metros

In [50]:
top10_metros = metros_df2.sort_values(by = ['metro_pop'], ascending = False).head(10)
top10_metros

,metro_code,metro_name,metro_type,metro_pop
621,35614,"New York-Jersey City-White Plains, NY-NJ",Metropolitan Division,11915488.0
508,31084,"Los Angeles-Long Beach-Glendale, CA",Metropolitan Division,10081570.0
166,16984,"Chicago-Naperville-Evanston, IL",Metropolitan Division,7175030.0
396,26420,"Houston-The Woodlands-Sugar Land, TX",Metropolitan Statistical Area,6884138.0
48,12060,"Atlanta-Sandy Springs-Alpharetta, GA",Metropolitan Statistical Area,5862424.0
215,19124,"Dallas-Plano-Irving, TX",Metropolitan Division,4903580.0
914,47894,"Washington-Arlington-Alexandria, DC-VA-MD-WV",Metropolitan Division,4901633.0
680,38060,"Phoenix-Mesa-Chandler, AZ",Metropolitan Statistical Area,4761603.0
730,40140,"Riverside-San Bernardino-Ontario, CA",Metropolitan Statistical Area,4560470.0
567,33460,"Minneapolis-St. Paul-Bloomington, MN-WI",Metropolitan Statistical Area,3573609.0


#### Of the largest metros, Chicago has the worst disparity for Black applicants. Lenders are 2.5 times more likely to deny Black applicants than similarly qualified White applicants. 

In [51]:
results_df5[(results_df5['metro_code'].isin(top10_metros['metro_code'])) & \
            (results_df5['variable_check'] == '5')]\
[['metro_code', 'metro_name', 'metro_pop', 'psuedo_rsquare', 'variable_name', 'p_value', 'odds_ratio']].\
sort_values(by = ['metro_pop', 'odds_ratio'], ascending = False)

,metro_code,metro_name,metro_pop,psuedo_rsquare,variable_name,p_value,odds_ratio
36,35614,"New York-Jersey City-White Plains, NY-NJ",11915488.0,0.212318,black,1.540452e-10,1.580497
2328,31084,"Los Angeles-Long Beach-Glendale, CA",10081570.0,0.211224,black,4.564155e-09,1.755397
2467,16984,"Chicago-Naperville-Evanston, IL",7175030.0,0.225103,black,8.646620e-37,2.483044
2468,16984,"Chicago-Naperville-Evanston, IL",7175030.0,0.225103,latino,1.587004e-16,1.591326
2119,26420,"Houston-The Woodlands-Sugar Land, TX",6884138.0,0.224035,black,5.975492e-28,2.041829
3060,12060,"Atlanta-Sandy Springs-Alpharetta, GA",5862424.0,0.247692,black,1.329923e-25,1.729919
2014,19124,"Dallas-Plano-Irving, TX",4903580.0,0.273443,black,2.003999e-19,2.101777
2017,19124,"Dallas-Plano-Irving, TX",4903580.0,0.273443,asian_cb,1.972840e-16,1.729214
385,47894,"Washington-Arlington-Alexandria, DC-VA-MD-WV",4901633.0,0.249535,black,1.402257e-19,2.073654
388,47894,"Washington-Arlington-Alexandria, DC-VA-MD-WV",4901633.0,0.249535,asian_cb,2.640166e-23,2.028950


#### Minneapolis is the only metro where all four race and ethnicites are more likely to be denied. 

In [52]:
results_df5[(results_df5['variable_check'] == '5')]['metro_name'].value_counts(dropna = False).head(5)

Minneapolis-St. Paul-Bloomington, MN-WI    4
Birmingham-Hoover, AL                      3
Indianapolis-Carmel-Anderson, IN           3
Cincinnati, OH-KY-IN                       3
Durham-Chapel Hill, NC                     3
Name: metro_name, dtype: int64

#### Key Metros
#### Chicago Results

In [53]:
results_df5[(results_df5['metro_code'] == '16984')][['metro_code', 'metro_name', 'metro_pop', 'metro_apps',
            'psuedo_rsquare', 'variable_name', 'p_value', 'odds_ratio']]

,metro_code,metro_name,metro_pop,metro_apps,psuedo_rsquare,variable_name,p_value,odds_ratio
2467,16984,"Chicago-Naperville-Evanston, IL",7175030.0,57155,0.225103,black,8.646620e-37,2.483044
2468,16984,"Chicago-Naperville-Evanston, IL",7175030.0,57155,0.225103,latino,1.587004e-16,1.591326
2469,16984,"Chicago-Naperville-Evanston, IL",7175030.0,57155,0.225103,native,3.414332e-01,1.547534
2470,16984,"Chicago-Naperville-Evanston, IL",7175030.0,57155,0.225103,asian_cb,2.562963e-07,1.388372


#### Minneapolis Results

In [55]:
results_df5[(results_df5['metro_code'] == '33460') & ((results_df5['variable_check'] == '4') |\
            (results_df5['variable_check'] == '5'))][['metro_code', 'metro_name', 'metro_pop', 'metro_apps',
            'psuedo_rsquare', 'variable_name', 'p_value', 'odds_ratio']]

,metro_code,metro_name,metro_pop,metro_apps,psuedo_rsquare,variable_name,p_value,odds_ratio
3475,33460,"Minneapolis-St. Paul-Bloomington, MN-WI",3573609.0,42162,0.266631,black,8.538529e-11,2.154326
3476,33460,"Minneapolis-St. Paul-Bloomington, MN-WI",3573609.0,42162,0.266631,latino,6.427513e-03,1.469225
3477,33460,"Minneapolis-St. Paul-Bloomington, MN-WI",3573609.0,42162,0.266631,native,3.944893e-02,2.067026
3478,33460,"Minneapolis-St. Paul-Bloomington, MN-WI",3573609.0,42162,0.266631,asian_cb,2.307718e-08,1.749404


#### Charolette Results

In [56]:
results_df5[(results_df5['metro_code'] == '16740') & ((results_df5['variable_check'] == '4') |\
            (results_df5['variable_check'] == '5'))][['metro_code', 'metro_name', 'metro_pop', 
            'psuedo_rsquare', 'variable_name', 'p_value', 'odds_ratio']]

,metro_code,metro_name,metro_pop,psuedo_rsquare,variable_name,p_value,odds_ratio
1736,16740,"Charlotte-Concord-Gastonia, NC-SC",2545560.0,0.243282,black,1.752335e-06,1.543780
1737,16740,"Charlotte-Concord-Gastonia, NC-SC",2545560.0,0.243282,latino,4.598104e-03,1.307147
1739,16740,"Charlotte-Concord-Gastonia, NC-SC",2545560.0,0.243282,asian_cb,1.757505e-07,1.744474


#### Largest disparites for Latinos

In [57]:
results_df5[(results_df5['variable_check'] == '5') & (results_df5['variable_name'] == 'latino')]\
[['metro_code', 'metro_name', 'metro_pop', 'psuedo_rsquare', 'variable_name', 'p_value', 'odds_ratio']].\
sort_values(by = ['odds_ratio'], ascending = False).head(5)

,metro_code,metro_name,metro_pop,psuedo_rsquare,variable_name,p_value,odds_ratio
12408,47380,"Waco, TX",268361.0,0.354818,latino,0.003080,3.291675
1841,24660,"Greensboro-High Point, NC",762063.0,0.248711,latino,0.000003,2.740411
2259,40060,"Richmond, VA",1269530.0,0.262993,latino,0.000006,2.676193
10321,31540,"Madison, WI",653725.0,0.248577,latino,0.000884,2.576313
4447,13820,"Birmingham-Hoover, AL",1085330.0,0.245919,latino,0.000088,2.411361


#### Largest disparites for AAPI

In [58]:
results_df5[(results_df5['variable_check'] == '5') & (results_df5['variable_name'] == 'asian_cb')]\
[['metro_code', 'metro_name', 'metro_pop', 'psuedo_rsquare', 'variable_name', 'p_value', 'odds_ratio']].\
sort_values(by = ['odds_ratio'], ascending = False).head(5)

,metro_code,metro_name,metro_pop,psuedo_rsquare,variable_name,p_value,odds_ratio
4240,38940,"Port St. Lucie, FL",472012.0,0.258088,asian_cb,5.200978e-04,3.086573
9397,40484,"Rockingham County-Strafford County, NH",435483.0,0.209597,asian_cb,2.149115e-04,2.962371
2261,40060,"Richmond, VA",1269530.0,0.262993,asian_cb,2.973412e-07,2.673358
18925,22220,"Fayetteville-Springdale-Rogers, AR",514259.0,0.230162,asian_cb,3.072155e-04,2.655565
2052,41700,"San Antonio-New Braunfels, TX",2468193.0,0.247725,asian_cb,1.727325e-12,2.560846


#### Largest disparites for Native American applicants

In [59]:
results_df5[(results_df5['variable_check'] == '5') & (results_df5['variable_name'] == 'native')]\
[['metro_code', 'metro_name', 'metro_pop', 'psuedo_rsquare', 'variable_name', 'p_value', 'odds_ratio']].\
sort_values(by = ['odds_ratio'], ascending = False).head(5)

,metro_code,metro_name,metro_pop,psuedo_rsquare,variable_name,p_value,odds_ratio
3477,33460,"Minneapolis-St. Paul-Bloomington, MN-WI",3573609.0,0.266631,native,0.039449,2.067026


#### Largest disparites for Black applicants

In [60]:
results_df5[(results_df5['variable_check'] == '5') & (results_df5['variable_name'] == 'black')]\
[['metro_code', 'metro_name', 'metro_pop', 'psuedo_rsquare', 'variable_name', 'p_value', 'odds_ratio']].\
sort_values(by = ['odds_ratio'], ascending = False).head(5)

,metro_code,metro_name,metro_pop,psuedo_rsquare,variable_name,p_value,odds_ratio
18922,22220,"Fayetteville-Springdale-Rogers, AR",514259.0,0.230162,black,7.221820e-03,3.098998
11143,19430,"Dayton-Kettering, OH",803543.0,0.279138,black,3.058314e-04,2.882675
5100,36420,"Oklahoma City, OK",1382841.0,0.270741,black,2.024550e-06,2.840909
2258,40060,"Richmond, VA",1269530.0,0.262993,black,5.007157e-08,2.720363
5169,44700,"Stockton, CA",742603.0,0.196393,black,1.838385e-04,2.621905


#### Largest disparites for Native American applicants

In [62]:
results_df5[(results_df5['variable_check'] == '5') & (results_df5['variable_name'] == 'native')]\
[['metro_code', 'metro_name', 'metro_pop', 'psuedo_rsquare', 'variable_name', 'p_value', 'odds_ratio']].\
sort_values(by = ['odds_ratio'], ascending = False).head(5)

,metro_code,metro_name,metro_pop,psuedo_rsquare,variable_name,p_value,odds_ratio
3477,33460,"Minneapolis-St. Paul-Bloomington, MN-WI",3573609.0,0.266631,native,0.039449,2.067026


#### Summary of findings by race and ethnicity
- Black applciants are more likely to be denied in 71 metros
- Latinos in 39 metros
- Asian/Pacific Islander in 55
- Native American in 1 metro

In [63]:
metro_race_bd = pd.pivot_table(results_df5, index = ['variable_name'], columns = ['variable_check'], 
                               values = ['metro_name'], aggfunc = 'count', fill_value = 0).reset_index()

metro_race_bd.columns = metro_race_bd.columns.droplevel(0)
metro_race_bd.columns.name = None
metro_race_bd.columns = ['Race/Ethnicity', 'Missing', 'Not Sig', 'More Apps', 'Small Disparity', 'Disparity']

metro_race_bd

,Race/Ethnicity,Missing,Not Sig,More Apps,Small Disparity,Disparity
0,asian_cb,0,61,2,10,55
1,black,1,48,6,2,71
2,latino,0,76,2,11,39
3,native,2,110,15,0,1


### 12. Write out results
#### Add definitions for output

In [64]:
lookup_dict = {'results_flag': ['1', '2', '3', '3', '3', '3', '3'],
               'variable_check': [np.nan, np.nan, '1', '2', '3', '4', '5'],
               'reliable_note': ['No results', 'Not enough variance in variables', 
                                 'Not statistically significant', 'Not statistically significant', 
                                 'Not enough applications', 'Doesn\'t meet level of disparity',
                                 'Statistically significant disparity']}

lookup_df = pd.DataFrame(data=lookup_dict)

lookup_df

,results_flag,variable_check,reliable_note
0,1,NaN,No results
1,2,NaN,Not enough variance in variables
2,3,1,Not statistically significant
3,3,2,Not statistically significant
4,3,3,Not enough applications
5,3,4,Doesn't meet level of disparity
6,3,5,Statistically significant disparity


In [66]:
results_df6 = pd.merge(results_df4, lookup_df, how = 'left', on = ['results_flag', 'variable_check'])

results_df6.loc[(results_df6['variable_check'] == '4') | (results_df6['variable_check'] == '5'),
                'is_reliable'] = True

results_df6.loc[(results_df6['variable_check'] != '4') & (results_df6['variable_check'] != '5'),
                'is_reliable'] = False

results_df6['odds_ratio_rd'] = results_df6['odds_ratio'].round(1)

#### Clean results for all metros:

In [67]:
races_replace = {'black': 'Black', 'latino': 'Latino', 'native': 'Native American', 'asian_cb': 'AAPI'}

results_df7 = results_df6[(results_df6['variable_name'].isin(races))]\
[['metro_code', 'metro_name', 'metro_pop', 'metro_apps', 'metro_type', 'variable_name', 'total_count', 'loan', 
  'denied', 'is_reliable', 'reliable_note', 'odds_ratio_rd']].rename(columns = {'odds_ratio_rd': 'odds_ratio'})

results_df8 = results_df7.replace(races_replace)

results_df8.sample(5, random_state = 303)

,metro_code,metro_name,metro_pop,metro_apps,metro_type,variable_name,total_count,loan,denied,is_reliable,reliable_note,odds_ratio
2780,36084,"Oakland-Berkeley-Livermore, CA",2799005.0,22676,Metropolitan Division,Black,655.0,591.0,64.0,True,Statistically significant disparity,1.7
30952,17380,"Cleveland, MS",31848.0,75,Micropolitan Statistical Area,Black,10.0,8.0,2.0,False,No results,NaN
18924,22220,"Fayetteville-Springdale-Rogers, AR",514259.0,5443,Metropolitan Statistical Area,Native American,49.0,46.0,3.0,False,Not statistically significant,1.5
26016,13780,"Binghamton, NY",241874.0,1070,Metropolitan Statistical Area,Latino,23.0,23.0,0.0,False,No results,NaN
29081,24820,"Greenville, OH",51513.0,159,Micropolitan Statistical Area,Black,1.0,1.0,0.0,False,No results,NaN


In [71]:
results_df8.to_csv('../../findings/metro_findings/1_metro_findings_200823.csv', index = False)